In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
from transformers import BertTokenizer

# GoEmotions

In [19]:
emotion_labels = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 
                  'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 
                  'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 
                  'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 
                  'relief', 'remorse', 'sadness', 'surprise', 'neutral']

In [26]:
test = torch.load("../data/goemotions-test.pth") # TensorDataset: (input_ids, attention_mask, labels)
labels = test.tensors[2].numpy()
labels[:5]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [28]:
probs = np.load("goemotions-probs.npy")
probs[:5]

array([[1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 0, 0, 1],
       [1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 0, 0, 1],
       [1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 0, 0, 1],
       [1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 0, 0, 1],
       [1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 0, 0, 1]])

In [33]:
misclassified = np.where(np.any(preds != labels, axis=1))[0]
print(f"Number of misclassified samples: {len(misclassified)}")
for index in misclassified[:5]:
    # print("Text:", test[index][0])
    print(f"Actual Emotions: {[emotion_labels[i] for i, label in enumerate(labels[index]) if label == 1]}")
    print(f"Predicted Emotions: {[emotion_labels[i] for i, label in enumerate(preds[index]) if label == 1]}\n")
    print()

Number of misclassified samples: 21123
Actual Emotions: ['excitement']
Predicted Emotions: ['admiration', 'amusement', 'annoyance', 'approval', 'curiosity', 'disappointment', 'disapproval', 'gratitude', 'joy', 'love', 'optimism', 'realization', 'neutral']


Actual Emotions: ['pride']
Predicted Emotions: ['admiration', 'amusement', 'annoyance', 'approval', 'curiosity', 'disappointment', 'disapproval', 'gratitude', 'joy', 'love', 'optimism', 'realization', 'neutral']


Actual Emotions: ['joy']
Predicted Emotions: ['admiration', 'amusement', 'annoyance', 'approval', 'curiosity', 'disappointment', 'disapproval', 'gratitude', 'joy', 'love', 'optimism', 'realization', 'neutral']


Actual Emotions: ['neutral']
Predicted Emotions: ['admiration', 'amusement', 'annoyance', 'approval', 'curiosity', 'disappointment', 'disapproval', 'gratitude', 'joy', 'love', 'optimism', 'realization', 'neutral']


Actual Emotions: ['neutral']
Predicted Emotions: ['admiration', 'amusement', 'annoyance', 'approval'

In [34]:
from sklearn.metrics import (hamming_loss, accuracy_score, jaccard_score,
                             precision_score, recall_score, f1_score,
                             roc_auc_score, average_precision_score, 
                             classification_report)

print("Classification Report:\n")
print(classification_report(labels, preds, target_names=emotion_labels))

Classification Report:

                precision    recall  f1-score   support

    admiration       0.08      1.00      0.15      1675
     amusement       0.04      1.00      0.08       873
         anger       0.00      0.00      0.00       835
     annoyance       0.07      1.00      0.13      1422
      approval       0.08      1.00      0.15      1729
        caring       0.00      0.00      0.00       605
     confusion       0.00      0.00      0.00       707
     curiosity       0.05      1.00      0.09       953
        desire       0.00      0.00      0.00       383
disappointment       0.04      1.00      0.08       835
   disapproval       0.06      1.00      0.11      1175
       disgust       0.00      0.00      0.00       570
 embarrassment       0.00      0.00      0.00       258
    excitement       0.00      0.00      0.00       513
          fear       0.00      0.00      0.00       353
     gratitude       0.05      1.00      0.10      1135
         grief       0.

/home/ecal/.local/share/virtualenvs/ai/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ecal/.local/share/virtualenvs/ai/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
exact_match_accuracy = accuracy_score(labels, preds)
print(f"Exact Match Accuracy: {exact_match_accuracy:.4f}")

averagePrecision = average_precision_score(labels, probs, average='macro')
print(f"Average Precision: {averagePrecision:.4f}")

Exact Match Accuracy: 0.0000
Average Precision: 0.0421


In [38]:
hammingLoss = hamming_loss(labels, preds)
print("Hamming Loss: {:.4f}".format(hammingLoss))

exactMatchRatio = accuracy_score(labels, preds)
print("Exact Match Ratio: {:.4f}".format(exactMatchRatio))

jaccardSimilarity = jaccard_score(labels, preds, average='samples')
print("Jaccard Similarity: {:.4f}\n".format(jaccardSimilarity))

Hamming Loss: 0.4432
Exact Match Ratio: 0.0000
Jaccard Similarity: 0.0675



In [39]:
print(f"{'Emotion':<20} {'ROC-AUC':<10} {'PR-AUC':<10}")
print("-" * 40)

for i, label in enumerate(emotion_labels):
    roc_auc = roc_auc_score(labels[:, i], probs[:, i])
    pr_auc = average_precision_score(labels[:, i], probs[:, i])
    print(f"{label:<20} {roc_auc:.4f}     {pr_auc:.4f}")

Emotion              ROC-AUC    PR-AUC    
----------------------------------------
admiration           0.5000     0.0793
amusement            0.5000     0.0413
anger                0.5000     0.0395
annoyance            0.5000     0.0673
approval             0.5000     0.0819
caring               0.5000     0.0286
confusion            0.5000     0.0335
curiosity            0.5000     0.0451
desire               0.5000     0.0181
disappointment       0.5000     0.0395
disapproval          0.5000     0.0556
disgust              0.5000     0.0270
embarrassment        0.5000     0.0122
excitement           0.5000     0.0243
fear                 0.5000     0.0167
gratitude            0.5000     0.0537
grief                0.5000     0.0030
joy                  0.5000     0.0397
love                 0.5000     0.0386
nervousness          0.5000     0.0088
optimism             0.5000     0.0381
pride                0.5000     0.0068
realization          0.5000     0.0420
relief             